In [ ]:
# ! pip install duckduckgo-search beautifulsoup4 requests
# ! pip install langchain duckduckgo-search ollama

In [2]:
from duckduckgo_search import DDGS
import requests
from bs4 import BeautifulSoup
import subprocess

## 1. Fazer a busca no DuckDuckGo

In [3]:
def buscar_links(query, max_results=5):
    with DDGS() as ddgs:
        resultados = ddgs.text(query)
        return [r["href"] for r in resultados][:max_results]


## 2. Extrair conteúdo da página

In [4]:
def extrair_texto(url):
    try:
        resp = requests.get(url, timeout=10)
        soup = BeautifulSoup(resp.content, "html.parser")
        return ' '.join(p.get_text() for p in soup.find_all('p'))
    except Exception as e:
        return f"[Erro ao acessar {url}: {e}]"

## 3. Enviar para o Ollama local com o modelo llama3

In [8]:

def perguntar_ollama(prompt, model="gemma3:12b"):
    process = subprocess.Popen(
        ["ollama", "run", model],
        stdin=subprocess.PIPE,
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE,
        text=True
    )
    resposta, erro = process.communicate(input=prompt)
    return resposta.strip()

## 4. Pipeline completa

In [9]:
def agente_viagem(query):
    print(f"🔍 Buscando por: {query}")
    links = buscar_links(query)
    print(f"🔗 Links encontrados: {links}\n")

    textos = [extrair_texto(link) for link in links]
    texto_compilado = '\n\n'.join(textos[:3])  # limite para não sobrecarregar

    prompt = (
        f"Considere o texto a seguir com informações sobre viagens para '{query}':\n\n"
        f"{texto_compilado}\n\n"
        "Faça um resumo dos melhores destinos e compare preços, se possível."
    )

    resposta = perguntar_ollama(prompt)
    print(f"\n🧠 Resumo do modelo:\n{resposta}")

In [ ]:
agente_viagem("melhores destinos para viajar saindo de Natal site: https://www.clubtravel.ie/")


🔍 Buscando por: melhores destinos para viajar saindo de Natal site: https://www.clubtravel.ie/
🔗 Links encontrados: []

